In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn import set_config; set_config(display='diagram')
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
from kidney_kids.data import get_preproc_data

### load data, split it and clean it for preoprocessing, retrieve X_train, X_test, y_train, y_test

In [7]:
path = '../raw_data/kidney_disease.csv'
url = "https://storage.googleapis.com/kidney_disaese/raw_data/kidney_disease.csv"
def get_cleaned_data(path=path):
    '''load data from csv
    and use cleaning fct to clean them'''
    df = pd.read_csv(path)
    y = df['classification']
    X = df.drop(columns= {'classification', 'id'})

    X = replacing_numerical_features(X)
    X,y = replacing_binary_features(X,y)

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_cleaned_data("https://storage.googleapis.com/kidney_disaese/raw_data/kidney_disease.csv")
X_train.columns


Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane'],
      dtype='object')

### helper functions to clean the data

In [8]:
def replacing_numerical_features(X):
    '''cleaning: strips \t at beginning of number and replaces ? with nan values'''
    X['pcv'] = X['pcv'].str.lstrip('\t')
    X['pcv'] = X['pcv'].replace(to_replace='?',value=np.nan).astype(float)
    X['wc'] = X['wc'].str.lstrip('\t')
    X['wc'] = X['wc'].replace(to_replace='?',value=np.nan).astype(float)
    X['rc'] = X['rc'].str.lstrip('\t')
    X['rc'] = X['rc'].replace(to_replace='?',value=np.nan).astype(float)
    return X


In [9]:
def replacing_binary_features(X,y):
    '''encoding: replacing Yes --> 1 no --> 0'''
    X[['htn','dm','cad','pe','ane']] = X[['htn','dm','cad','pe','ane']].replace(to_replace={'yes':1,'no':0})
    X[['rbc','pc']] = X[['rbc','pc']].replace(to_replace={'abnormal':1,'normal':0})
    X[['pcc','ba']] = X[['pcc','ba']].replace(to_replace={'present':1,'notpresent':0})
    X[['appet']] = X[['appet']].replace(to_replace={'good':2,'poor':1,'no':0})
    ## replacing t_values to 0 or 1, by assuming it s close to 0 or 1, respectively
    X['cad'] = X['cad'].replace(to_replace='\tno',value=0)
    X['dm'] = X['dm'].replace(to_replace={'\tno':0,'\tyes':1,' yes':1})

    #encoding the target:
    y= y.replace(to_replace={'ckd':1,'notckd':0, 'ckd\t': 1}).astype(int)
    return X,y

### preprocessing of the data

In [15]:
def preproc(X_train):
    '''takes in a df with column names. returns preprocessed data as numpy array with 24 columns'''
    # creating feat_lists for pipeline
    feat_binary = ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']
    feat_ordered = ['sg', 'al', 'su']
    feat_continuous = ['age', 'bp', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc',
       'rc']

 

    ordered_transformer = Pipeline([
                                ('cat_imputer', SimpleImputer(strategy='most_frequent')),
                                ('mm_scaler', MinMaxScaler())
                                ])

    binary_transformer = Pipeline([
                                ('cat_imputer', SimpleImputer(strategy='most_frequent'))
                                ])

    cont_transformer = Pipeline([
                                ('num_imputer', SimpleImputer()),
                                ('mm_scaler', MinMaxScaler())
                                ])

    preproc_pipe = ColumnTransformer([
                                        ('ord_trans', ordered_transformer, feat_ordered),
                                        ('bin_trans', binary_transformer, feat_binary),
                                        ('cont_trans', cont_transformer, feat_continuous)
                                    ])


    X_proc = preproc_pipe.fit_transform(X_train)

    return X_proc
X_preproc = preproc(X_train)
X_preproc


array([[0.25      , 0.        , 0.        , ..., 0.4       , 0.23109244,
        0.18644068],
       [0.75      , 0.4       , 0.        , ..., 0.44444444, 0.24578036,
        0.44376683],
       [0.75      , 0.        , 0.        , ..., 0.71111111, 0.15546218,
        0.6440678 ],
       ...,
       [0.75      , 0.        , 0.        , ..., 0.73333333, 0.23109244,
        0.44067797],
       [0.25      , 0.4       , 0.        , ..., 0.57777778, 0.19747899,
        0.42372881],
       [0.25      , 0.2       , 0.2       , ..., 0.6       , 0.24578036,
        0.44376683]])

In [11]:
model = RandomForestClassifier()
model.fit(preproc(X_train), y_train)
numpy_array = np.array([0.75, 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.29885057, 0.23076923, 0.13034188, 0.12195122, 0.00529801, 0.84858044, 0.01797753, 0.97692308, 0.65, 0.31818182, 0.44067797]).reshape(1, -1)
model.predict(numpy_array)  

array([0])

### Grid search on Forestmodel

In [29]:
'''def forest_model(X_proc,y_train):
    #create the model, do the gridsearch
    #and return fitted model with best params
    rfc=RandomForestClassifier()

    param_grid = {
    'n_estimators': [100, 300, 500],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [3,5,7,10,15],
    'min_samples_split' : [2, 3, 5, 7]
    }

    
    search = GridSearchCV(rfc, param_grid=param_grid, scoring='recall')
    result = search.fit(X_proc,y_train)

    df = pd.DataFrame(result.cv_results_)

    return [result.best_estimator_, df, result.best_params_]
model, df, best_prams = forest_model(X_preproc, y_train)
model'''

"def forest_model(X_proc,y_train):\n    #create the model, do the gridsearch\n    #and return fitted model with best params\n    rfc=RandomForestClassifier()\n\n    param_grid = {\n    'n_estimators': [100, 300, 500],\n    'criterion' : ['gini', 'entropy'],\n    'max_depth' : [3,5,7,10,15],\n    'min_samples_split' : [2, 3, 5, 7]\n    }\n\n    \n    search = GridSearchCV(rfc, param_grid=param_grid, scoring='recall')\n    result = search.fit(X_proc,y_train)\n\n    df = pd.DataFrame(result.cv_results_)\n\n    return [result.best_estimator_, df, result.best_params_]\nmodel, df, best_prams = forest_model(X_preproc, y_train)\nmodel"

In [13]:
#model = RandomForestClassifier()
model.fit(X_preproc,y_train)
model.predict(preproc(X_test))


array([0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1])

In [14]:
pd.DataFrame(preproc(X_test)).head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.500000,0.031830,0.168317,0.001325,0.979381,0.042697,0.815534,0.766667,0.530769,0.555556
1,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.333333,0.200015,0.215413,0.044230,0.904443,0.060915,0.611650,0.833333,0.430769,0.472222
2,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.333333,0.023873,0.158416,0.000000,0.945017,0.051685,0.883495,0.566667,0.276923,0.888889
3,0.5,0.00,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.500000,0.228117,0.148515,0.025166,0.931271,0.020225,0.398058,0.400000,0.623077,0.361111
4,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.166667,0.021220,0.059406,0.005298,0.904443,0.060915,0.466019,0.600000,0.346154,0.545960


### function testing for api

In [17]:
def plot_df(feat1, feat2):
    X_train = get_cleaned_data()[0]
    df = get_preproc_data(X_train)
    df['target'] = get_cleaned_data()[2]
    df[feat1, feat2, 'target']

    return df[feat1, feat2, 'target']

In [25]:
df = get_preproc_data(X_train)
df



,sg,al,su,rbc,pc,pcc,ba,htn,dm,cad,...,bp,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc
0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.230769,0.294872,0.098845,0.033543,0.610169,0.043836,0.401361,0.400000,0.231092,0.186441
1,0.75,0.4,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.384615,0.252137,0.137356,0.044025,0.573689,0.054784,0.476190,0.444444,0.245780,0.443767
2,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.153846,0.220085,0.093710,0.004193,0.610169,0.068493,0.931973,0.711111,0.155462,0.644068
3,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.076923,0.170940,0.065469,0.006289,0.644068,0.065753,0.687075,0.777778,0.352941,0.559322
4,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.230769,0.143162,0.103979,0.002096,0.593220,0.068493,0.925170,0.955556,0.319328,0.491525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,0.25,0.6,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.153846,0.115385,0.473684,0.306080,0.525424,0.139726,0.272109,0.288889,0.050420,0.000000
316,0.25,0.4,0.4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.230769,0.270660,0.142910,0.062893,0.542373,0.046575,0.673469,0.688889,0.302521,0.355932
317,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.076923,0.179487,0.065469,0.006289,0.779661,0.021918,0.768707,0.733333,0.231092,0.440678
318,0.25,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.230769,0.179487,0.062901,0.020964,0.573689,0.054784,0.578231,0.577778,0.197479,0.423729


## Bausteine für die api

### PredictEndpoint: erste möglichkeit ( weiß nicht, ob es fkt)

In [67]:
from urllib.parse import urlparse
from urllib.parse import parse_qs

url = 'https://www.example.com/some_path?some_key=6&age=9'
param_query = parse_qs(urlparse(url).query)
param_list = []
for i in param_query.values():
    param_list.append(int(i[0]))
param_list


[6, 9]

### zweite möglichkeit: retrieve all paramters passed to function as list

In [112]:
import inspect
def foo(a=0,b=9,c=4): 
    dict = {}
    args = inspect.getfullargspec(foo)
    arg_val_list = list(args.defaults)
    args_list = ['age', 'bp', 'sg']
    for i,j in zip(args_list, arg_val_list):
        dict[i] = [j]
    print(dict)
    df = pd.DataFrame.from_dict(dict)
    return df
foo()

{'age': [0], 'bp': [9], 'sg': [4]}


,age,bp,sg
0,0,9,4


### produce query for api request to test

In [111]:
names = ['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane']
values = [0.75, 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.29885057, 0.23076923, 0.13034188, 0.12195122, 0.00529801, 0.84858044, 0.01797753, 0.97692308, 0.65, 0.31818182, 0.44067797]
query=''
for i,j in zip(names, values):
    query = query + i + '=' + str(j) + '&'
query   

'age=0.75&bp=0.0&sg=0.0&al=0.0&su=0.0&rbc=0.0&pc=0.0&pcc=0.0&ba=0.0&bgr=0.0&bu=2.0&sc=0.0&sod=0.0&pot=0.29885057&hemo=0.23076923&pcv=0.13034188&wc=0.12195122&rc=0.00529801&htn=0.84858044&dm=0.01797753&cad=0.97692308&appet=0.65&pe=0.31818182&ane=0.44067797&'

## ScatterEndpoint

In [26]:
    BUCKET_NAME = 'kidney_disaese'
    bucket=BUCKET_NAME
    MODEL_NAME = 'forest_model'
    client = storage.Client().bucket(bucket)

    storage_location = 'models/forest_model/v1/model.joblib'
    blob = client.blob(storage_location)
    blob.download_to_filename('model.joblib')
    model = joblib.load('model.joblib')

NameError: name 'storage' is not defined

In [ ]:
from io import BytesIO
from starlette.responses import StreamingResponse
...


@router.get("/graph/{id_file}", name="Return the graph obtained")
async def create_graph(id_file: str):
    data = HAR.createGraph(id_file)
    graph = HAR.scatterplot(data['dateTimes'], data['label'], "Time", "Activity")
    
    # create a buffer to store image data
    buf = BytesIO()
    graph.savefig(buf, format="png")
    buf.seek(0)
        
    return StreamingResponse(buf, media_type="image/png")